In [12]:
import psycopg2
import pandas as pd
from pandas import Series, DataFrame

In [2]:
connection = psycopg2.connect(database="01_dvdrental", user="postgres", password="qkrtkddus!1")
cursor = connection.cursor()

In [17]:
cursor.execute("select * from PRODUCT_GROUP")
DataFrame(cursor.fetchall(), columns = ["group_id", "group_name"])

,group_id,group_name
0,1,Smartphone
1,2,Laptop
2,3,Tablet


In [59]:
cursor.execute("select * from PRODUCT")
df = DataFrame(cursor.fetchall(), columns = ["product_id", "product_name", "price", "group_id"])
df = df.astype({'price':'int'})
df

,product_id,product_name,price,group_id
0,1,Microsoft Lumia,200,1
1,2,HTC One,400,1
2,3,Nexus,500,1
3,4,iPhone,900,1
4,5,HP Elite,1200,2
5,6,Lenovo Thinkpad,700,2
6,7,Sony VAIO,700,2
7,8,Dell Vostro,800,2
8,9,iPad,700,3
9,10,Kindle Fire,150,3


In [58]:
cursor.execute("SELECT *      , COUNT(*) OVER() FROM PRODUCT")
df = DataFrame(cursor.fetchall(), columns = ["product_id", "product_name", "price", "group_id", "count"])
df = df.astype({'price':'int'})
df

,product_id,product_name,price,group_id,count
0,1,Microsoft Lumia,200,1,11
1,2,HTC One,400,1,11
2,3,Nexus,500,1,11
3,4,iPhone,900,1,11
4,5,HP Elite,1200,2,11
5,6,Lenovo Thinkpad,700,2,11
6,7,Sony VAIO,700,2,11
7,8,Dell Vostro,800,2,11
8,9,iPad,700,3,11
9,10,Kindle Fire,150,3,11


In [57]:
cursor.execute("SELECT A.PRODUCT_NAME , A.PRICE , B.GROUP_NAME , AVG (A.PRICE) OVER (PARTITION BY B.GROUP_NAME) FROM PRODUCT A INNER JOIN PRODUCT_GROUP B ON (A.GROUP_ID = B.GROUP_ID);")
df = DataFrame(cursor.fetchall(), columns = ["product_name", "price", "group_name", "avg"])
df = df.astype({'avg':'int','price': 'int'})
df

,product_name,price,group_name,avg
0,HP Elite,1200,Laptop,850
1,Lenovo Thinkpad,700,Laptop,850
2,Sony VAIO,700,Laptop,850
3,Dell Vostro,800,Laptop,850
4,Microsoft Lumia,200,Smartphone,500
5,HTC One,400,Smartphone,500
6,Nexus,500,Smartphone,500
7,iPhone,900,Smartphone,500
8,iPad,700,Tablet,350
9,Kindle Fire,150,Tablet,350


In [55]:
cursor.execute("select 	A.PRODUCT_NAME 	, B.GROUP_NAME 	, A.PRICE , ROW_NUMBER () OVER ( PARTITION BY B.GROUP_NAME ORDER BY A.PRICE) FROM PRODUCT A INNER JOIN PRODUCT_GROUP B ON (A.GROUP_ID = B.GROUP_ID);")
df = DataFrame(cursor.fetchall(), columns = ["product_name", "group_name", "price", "row_number"])
df = df.astype({'price':'int'})
df

,product_name,group_name,price,row_number
0,Sony VAIO,Laptop,700,1
1,Lenovo Thinkpad,Laptop,700,2
2,Dell Vostro,Laptop,800,3
3,HP Elite,Laptop,1200,4
4,Microsoft Lumia,Smartphone,200,1
5,HTC One,Smartphone,400,2
6,Nexus,Smartphone,500,3
7,iPhone,Smartphone,900,4
8,Kindle Fire,Tablet,150,1
9,Samsung Galaxy Tab,Tablet,200,2


In [54]:
cursor.execute("select 	A.PRODUCT_NAME 	, B.GROUP_NAME 	, A.PRICE , RANK () OVER ( PARTITION BY B.GROUP_NAME ORDER BY A.PRICE) FROM PRODUCT A INNER JOIN PRODUCT_GROUP B ON (A.GROUP_ID = B.GROUP_ID);")
df = DataFrame(cursor.fetchall(), columns = ["product_name", "group_name", "price", "rank"])
df = df.astype({'price':'int'})
df

,product_name,group_name,price,rank
0,Sony VAIO,Laptop,700,1
1,Lenovo Thinkpad,Laptop,700,1
2,Dell Vostro,Laptop,800,3
3,HP Elite,Laptop,1200,4
4,Microsoft Lumia,Smartphone,200,1
5,HTC One,Smartphone,400,2
6,Nexus,Smartphone,500,3
7,iPhone,Smartphone,900,4
8,Kindle Fire,Tablet,150,1
9,Samsung Galaxy Tab,Tablet,200,2


In [50]:
connection.commit()

In [53]:
cursor.execute("select 	A.PRODUCT_NAME 	, B.GROUP_NAME 	, A.PRICE , dense_rank () OVER ( PARTITION BY B.GROUP_NAME ORDER BY A.PRICE) FROM PRODUCT A INNER JOIN PRODUCT_GROUP B ON (A.GROUP_ID = B.GROUP_ID);")
df = DataFrame(cursor.fetchall(), columns = ["product_name", "group_name", "price", "dense_rank"])
df = df.astype({'price':'int'})
df

,product_name,group_name,price,dense_rank
0,Sony VAIO,Laptop,700,1
1,Lenovo Thinkpad,Laptop,700,1
2,Dell Vostro,Laptop,800,2
3,HP Elite,Laptop,1200,3
4,Microsoft Lumia,Smartphone,200,1
5,HTC One,Smartphone,400,2
6,Nexus,Smartphone,500,3
7,iPhone,Smartphone,900,4
8,Kindle Fire,Tablet,150,1
9,Samsung Galaxy Tab,Tablet,200,2


In [30]:
cursor.execute("select A.PRODUCT_NAME, B.GROUP_NAME, A.PRICE , FIRST_VALUE (A.PRICE) over (PARTITION BY B.GROUP_NAME ORDER BY A.PRICE ) AS LOWEST_PRICE_PER_GROUP FROM PRODUCT A INNER JOIN PRODUCT_GROUP B ON (A.GROUP_ID = B.GROUP_ID); ")
df = DataFrame(cursor.fetchall(), columns = ["product_name", "group_name", "price", "lowest_price_per_group"])
df = df.astype({'lowest_price_per_group':'int', 'price':'int'})
df

,product_name,group_name,price,lowest_price_per_group
0,Sony VAIO,Laptop,700,700
1,Lenovo Thinkpad,Laptop,700,700
2,Dell Vostro,Laptop,800,700
3,HP Elite,Laptop,1200,700
4,Microsoft Lumia,Smartphone,200,200
5,HTC One,Smartphone,400,200
6,Nexus,Smartphone,500,200
7,iPhone,Smartphone,900,200
8,Kindle Fire,Tablet,150,150
9,Samsung Galaxy Tab,Tablet,200,150


In [32]:
cursor.execute("select A.PRODUCT_NAME, B.GROUP_NAME, A.PRICE , LAST_VALUE (A.PRICE) over (PARTITION BY B.GROUP_NAME ORDER BY A.PRICE RANGE BETWEEN UNBOUNDED preceding AND UNBOUNDED FOLLOWING) AS HIGHEST_PRICE_PER_GROUP FROM PRODUCT A INNER JOIN PRODUCT_GROUP B ON (A.GROUP_ID = B.GROUP_ID);")
df = DataFrame(cursor.fetchall(), columns = ["product_name", "group_name", "price", "upper_price_per_group"])
df = df.astype({'upper_price_per_group':'int', 'price':'int'})
df

,product_name,group_name,price,upper_price_per_group
0,Sony VAIO,Laptop,700,1200
1,Lenovo Thinkpad,Laptop,700,1200
2,Dell Vostro,Laptop,800,1200
3,HP Elite,Laptop,1200,1200
4,Microsoft Lumia,Smartphone,200,900
5,HTC One,Smartphone,400,900
6,Nexus,Smartphone,500,900
7,iPhone,Smartphone,900,900
8,Kindle Fire,Tablet,150,700
9,Samsung Galaxy Tab,Tablet,200,700


In [42]:
cursor.execute("SELECT A.PRODUCT_NAME, B.GROUP_NAME, A.PRICE, LAG (A.PRICE, 1) OVER (PARTITION BY B.GROUP_NAME ORDER BY A.PRICE ) AS PREV_PRICE , A.PRICE - LAG (A.PRICE, 1) OVER (PARTITION BY B.GROUP_NAME ORDER BY A.PRICE ) AS CUR_PREV_DIFF FROM PRODUCT A INNER JOIN PRODUCT_GROUP B ON (A.GROUP_ID = B.GROUP_ID);")
df = DataFrame(cursor.fetchall(), columns = ["product_name", "group_name", "price", "prev_price", "cur_prev_diff"])
df = df.astype({'price':'int'})
df

,product_name,group_name,price,prev_price,cur_prev_diff
0,Sony VAIO,Laptop,700,None,None
1,Lenovo Thinkpad,Laptop,700,700.00,0.00
2,Dell Vostro,Laptop,800,700.00,100.00
3,HP Elite,Laptop,1200,800.00,400.00
4,Microsoft Lumia,Smartphone,200,None,None
5,HTC One,Smartphone,400,200.00,200.00
6,Nexus,Smartphone,500,400.00,100.00
7,iPhone,Smartphone,900,500.00,400.00
8,Kindle Fire,Tablet,150,None,None
9,Samsung Galaxy Tab,Tablet,200,150.00,50.00


In [43]:
cursor.execute("SELECT A.PRODUCT_NAME, B.GROUP_NAME, A.PRICE , 			LEAD (A.PRICE, 1) OVER ( PARTITION BY B.GROUP_NAME ORDER BY A.PRICE ) AS NEXT_PRICE , A.PRICE - LEAD (A.PRICE, 1) OVER ( PARTITION BY B.GROUP_NAME ORDER BY A.PRICE )AS CUR_NEXT_DIFF FROM PRODUCT A INNER JOIN PRODUCT_GROUP B ON (A.GROUP_ID = B.GROUP_ID);")
df = DataFrame(cursor.fetchall(), columns = ["product_name", "group_name", "price", "prev_price", "cur_prev_diff"])
df = df.astype({'price':'int'})
df

,product_name,group_name,price,prev_price,cur_prev_diff
0,Sony VAIO,Laptop,700,700.00,0.00
1,Lenovo Thinkpad,Laptop,700,800.00,-100.00
2,Dell Vostro,Laptop,800,1200.00,-400.00
3,HP Elite,Laptop,1200,None,None
4,Microsoft Lumia,Smartphone,200,400.00,-200.00
5,HTC One,Smartphone,400,500.00,-100.00
6,Nexus,Smartphone,500,900.00,-400.00
7,iPhone,Smartphone,900,None,None
8,Kindle Fire,Tablet,150,200.00,-50.00
9,Samsung Galaxy Tab,Tablet,200,700.00,-500.00
